In [1]:
from sodapy import Socrata
import pandas as pd
import mysql.connector as mysql

In [3]:
#function to connect ot mysql 
def connect_db(db_name , host , user , password):
    try:
        db = mysql.connect(
            host = host,
            user = user,
            password = password,
            database = db_name
        )
        
        return db
    except:
        print("could not access the database")

In [16]:
def create_tables(cursor):
    cursor.execute("SHOW TABLES")
    current_tables = list(cursor)
    current_tables = list(map(lambda x:x[0] , current_tables))
    if "pd_cnts_snsrs" not in current_tables:
        Q1 = '''CREATE TABLE pd_cnts_snsrs(sensor_id INT PRIMARY KEY, 
            place VARCHAR(100), 
            lattitude FLOAT, 
            longitude FLOAT)
            '''
        
        cursor.execute(Q1)
        print("pd_cnts_snsrs table created")
    else:
        print("pd_cnts_snsrs table already exists")
        
    if "pd_cnts" not in current_tables:
        Q2 = '''CREATE TABLE pd_cnts(
        sensor_id INT , 
        pedestrian_count INT , 
        date_time DATETIME ,
        PRIMARY KEY(sensor_id , date_time))'''
        
        Q3 = "ALTER TABLE pd_cnts ADD FOREIGN KEY(sensor_id) REFERENCES pd_cnts_snsrs(sensor_id)"
        cursor.execute(Q2)
        cursor.execute(Q3)
        print("pd_cnts table created")
    else : 
        print("pd_cnts already exists")

In [17]:
def populate_sensors(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC" ):
    client = Socrata("data.melbourne.vic.gov.au", access_token)
    results = client.get("h57g-5234")
    results_df = pd.DataFrame.from_records(results)
    sensors = results_df[["sensor_id" , "sensor_description" , "latitude" , "longitude"]]
    # sensors.iloc[:,0] = sensors.sensor_id.astype("int")
    # sensors.iloc[:,1] = sensors.sensor_description.astype("str")
    # sensors.iloc[:,2] = sensors.latitude.astype("float")
    # sensors.iloc[:,3] = sensors.longitude.astype("float")
    sensors_list = [tuple(x) for x in sensors.to_numpy()]

    Q4 = "INSERT INTO pd_cnts_snsrs(sensor_id , place, lattitude , longitude) VALUES (%s , %s , %s , %s)"
    cursor.executemany(Q4 , sensors_list)
    print("successfully added data to pd_cnts_snsrs table ")
    return 1

In [18]:
def populate_sensors_data(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC"):
    client = Socrata("data.melbourne.vic.gov.au",access_token)
    results = client.get("d6mv-s43h")
    results_df = pd.DataFrame.from_records(results)
    sensor_data = results_df[["sensor_id"  , "total_of_directions" , "date_time"]]
    sensors_data_list = [tuple(x) for x in sensor_data.to_numpy()]
    
    Q5 = "INSERT INTO pd_cnts(sensor_id , pedestrian_count , date_time) VALUES (%s, %s , %s)"
    cursor.executemany(Q5 , sensors_data_list)
    print("successfully added data to pd_cnts table")

In [19]:
# db = connect_db("wordpress" , "wordpress.ckoizwonzrvu.us-east-2.rds.amazonaws.com" , "admin", "owlleadma26")
# cursor = db.cursor()
create_tables(cursor)
populate_sensors(cursor)
populate_sensors_data(cursor)

pd_cnts_snsrs table already exists
pd_cnts already exists
successfully added data to pd_cnts_snsrs table 
successfully added data to pd_cnts table


In [22]:
cursor.execute('''SELECT  pd_cnts_snsrs.place , AVG(pd_cnts.pedestrian_count) as avg_count
FROM pd_cnts
INNER JOIN pd_cnts_snsrs on pd_cnts.sensor_id = pd_cnts_snsrs.sensor_id
GROUP BY pd_cnts_snsrs.place 
ORDER BY avg_count DESC''')
ans = list(cursor)


In [26]:
cursor.execute("SHOW TABLES")
list(cursor)

[('pd_cnts',),
 ('pd_cnts_snsrs',),
 ('wp_actionscheduler_actions',),
 ('wp_actionscheduler_claims',),
 ('wp_actionscheduler_groups',),
 ('wp_actionscheduler_logs',),
 ('wp_commentmeta',),
 ('wp_comments',),
 ('wp_links',),
 ('wp_options',),
 ('wp_pmxi_files',),
 ('wp_pmxi_hash',),
 ('wp_pmxi_history',),
 ('wp_pmxi_images',),
 ('wp_pmxi_imports',),
 ('wp_pmxi_posts',),
 ('wp_pmxi_templates',),
 ('wp_postmeta',),
 ('wp_posts',),
 ('wp_term_relationships',),
 ('wp_term_taxonomy',),
 ('wp_termmeta',),
 ('wp_terms',),
 ('wp_usermeta',),
 ('wp_users',),
 ('wp_wpforms_tasks_meta',)]

In [17]:
cursor.execute("SELECT * from Sensors")
list(cursor)

[(1, 'Bourke Street Mall (North)', -37.8135, 144.965),
 (2, 'Bourke Street Mall (South)', -37.8138, 144.965),
 (3, 'Melbourne Central', -37.811, 144.964),
 (4, 'Town Hall (West)', -37.8149, 144.966),
 (5, 'Princes Bridge', -37.8187, 144.968),
 (6, 'Flinders Street Station Underpass', -37.8191, 144.966),
 (7, 'Birrarung Marr', -37.8186, 144.972),
 (8, 'Webb Bridge', -37.8229, 144.947),
 (9, 'Southern Cross Station', -37.8198, 144.951),
 (10, 'Victoria Point', -37.8188, 144.947),
 (11, 'Waterfront City', -37.8157, 144.94),
 (12, 'New Quay', -37.8146, 144.943),
 (13, 'Flagstaff Station', -37.8124, 144.957),
 (14, 'Sandridge Bridge', -37.8201, 144.963),
 (15, 'State Library', -37.8106, 144.964),
 (16, 'Australia on Collins', -37.8157, 144.965),
 (17, 'Collins Place (South)', -37.8136, 144.973),
 (18, 'Collins Place (North)', -37.8134, 144.973),
 (19, 'Chinatown-Swanston St (North)', -37.8124, 144.965),
 (20, 'Chinatown-Lt Bourke St (South)', -37.8117, 144.968),
 (21, 'Bourke St-Russell St 